In [7]:
import pandas as pd
import numpy as np

In [97]:
data_base_path = './result/'
od_path = data_base_path+'od_june.npy'
pu_count_path = data_base_path+'pu_count_june.csv'
zone_table_path = './data/taxi-data/taxi+_zone_lookup.csv'

In [98]:
zone_df = pd.read_csv(zone_table_path)
def translate_zone_code(zone_df,code):
    return zone_df[zone_df['LocationID']==code]['Zone'].item()

# Analysis on OD table

In [106]:
od = np.load(od_path)
pu_count_df = pd.read_csv(pu_count_path)

In [107]:
def get_topn_pu(n,pu_count_df):
    top_pu = pu_count_df[:n]
    return top_pu['id']

def get_topn_do_from_pu(n,pu_id,od):
    d = od[pu_id]
    return sorted(range(len(d)), key=lambda i: d[i])[-n:]
    
def create_empty_df(column_names):
    df = pd.DataFrame(columns=column_names)
    return df
        

In [108]:
def translate_do_names(do_ids):
    do_names = []
    for do_id in do_ids:
        do_names.append(translate_zone_code(zone_df,do_id))
    return do_names

def create_topn_od(n,pu_count_df,od):
    top_pu = get_topn_pu(n,pu_count_df)
    column_names = ['pu_id']+[i+1 for i in range(n)]
    top_df = create_empty_df(column_names)
    for i,pu_id in enumerate(top_pu):
        top_do = get_topn_do_from_pu(n,pu_id,od)
        top_do_names = translate_do_names(top_do)
        top_df.loc[i] = [translate_zone_code(zone_df,pu_id)]+ top_do_names
    return top_df

In [109]:
# top 10 places to pick up
n = 10
top10_df = create_topn_od(n,pu_count_df,od)

In [110]:
top10_df

,pu_id,1,2,3,4,5,6,7,8,9,10
0,Upper East Side South,Sutton Place/Turtle Bay North,Midtown North,Lenox Hill East,Yorkville West,Lincoln Square East,Lenox Hill West,Midtown Center,Midtown East,Upper East Side South,Upper East Side North
1,Midtown Center,Midtown East,Midtown North,Penn Station/Madison Sq West,Murray Hill,Times Sq/Theatre District,Union Sq,Midtown South,Upper East Side North,Midtown Center,Upper East Side South
2,Upper East Side North,Yorkville East,Upper West Side North,East Harlem South,Yorkville West,Upper West Side South,Midtown Center,Midtown East,Lenox Hill West,Upper East Side North,Upper East Side South
3,Midtown East,Union Sq,Times Sq/Theatre District,Penn Station/Madison Sq West,Gramercy,Lenox Hill West,Upper East Side North,Midtown Center,Midtown East,Upper East Side South,Murray Hill
4,Penn Station/Madison Sq West,East Chelsea,Gramercy,Midtown North,West Chelsea/Hudson Yards,Midtown East,Union Sq,Clinton East,Murray Hill,Times Sq/Theatre District,Midtown Center
5,JFK Airport,Midtown North,Baisley Park,Midtown East,Midtown Center,Midtown South,Murray Hill,Clinton East,NaN,Times Sq/Theatre District,JFK Airport
6,Times Sq/Theatre District,West Chelsea/Hudson Yards,Midtown East,Garment District,Midtown North,Clinton East,LaGuardia Airport,Lincoln Square East,Times Sq/Theatre District,Midtown Center,Penn Station/Madison Sq West
7,Clinton East,Upper West Side South,Penn Station/Madison Sq West,Clinton West,Midtown North,West Chelsea/Hudson Yards,Times Sq/Theatre District,Lincoln Square East,Midtown Center,East Chelsea,Clinton East
8,Murray Hill,Times Sq/Theatre District,Midtown South,Sutton Place/Turtle Bay North,Upper East Side South,Midtown East,Penn Station/Madison Sq West,Gramercy,Murray Hill,Union Sq,Midtown Center
9,Lincoln Square East,Lincoln Square West,Upper East Side North,Times Sq/Theatre District,Midtown Center,Midtown North,Lincoln Square East,Upper East Side South,Clinton East,Upper West Side North,Upper West Side South


In [96]:
top10_df

,pu_id,1,2,3,4,5,6,7,8,9,10
0,Upper East Side South,Upper West Side South,Sutton Place/Turtle Bay North,Lincoln Square East,Yorkville West,Lenox Hill East,Lenox Hill West,Midtown Center,Midtown East,Upper East Side South,Upper East Side North
1,Upper East Side North,Midtown Center,East Harlem South,Midtown East,Upper West Side North,Upper West Side South,Yorkville East,Yorkville West,Lenox Hill West,Upper East Side South,Upper East Side North
2,Midtown Center,Clinton East,Midtown North,Murray Hill,Penn Station/Madison Sq West,Midtown South,Union Sq,Times Sq/Theatre District,Upper East Side North,Midtown Center,Upper East Side South
3,Midtown East,Union Sq,Times Sq/Theatre District,Penn Station/Madison Sq West,Gramercy,Midtown Center,Upper East Side North,Lenox Hill West,Midtown East,Upper East Side South,Murray Hill
4,Times Sq/Theatre District,Murray Hill,Garment District,West Chelsea/Hudson Yards,Midtown East,Midtown North,Lincoln Square East,Times Sq/Theatre District,Clinton East,Midtown Center,Penn Station/Madison Sq West
5,Penn Station/Madison Sq West,East Chelsea,Midtown North,Gramercy,West Chelsea/Hudson Yards,Midtown East,Clinton East,Union Sq,Murray Hill,Times Sq/Theatre District,Midtown Center
6,Clinton East,Lincoln Square West,Penn Station/Madison Sq West,Midtown North,Clinton West,West Chelsea/Hudson Yards,Lincoln Square East,Times Sq/Theatre District,Midtown Center,East Chelsea,Clinton East
7,Murray Hill,Upper East Side South,Times Sq/Theatre District,Midtown South,Sutton Place/Turtle Bay North,Midtown East,Penn Station/Madison Sq West,Gramercy,Murray Hill,Midtown Center,Union Sq
8,Union Sq,Midtown South,Gramercy,Greenwich Village North,West Village,Union Sq,East Village,East Chelsea,Midtown Center,Penn Station/Madison Sq West,Murray Hill
9,Lincoln Square East,Times Sq/Theatre District,Midtown Center,Midtown North,Upper East Side North,Lincoln Square West,Upper East Side South,Clinton East,Lincoln Square East,Upper West Side North,Upper West Side South
